In [1]:
import json
import joblib
import pickle
import pandas as pd
import numpy as np
# import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('data/train.csv')

# transforming Date column in datetime
train['Date']=pd.to_datetime(train['Date'], infer_datetime_format=True)

# auxiliar Date feature (for temporal analyzis)
train['referencia'] = train['Date'].dt.year*100 + train['Date'].dt.month

# according to Dr Wilson's instructions: 
# "If there was a search and the outcome linked to object of search has not been written please consider it to be False"
train['Outcome linked to object of search'] = train['Outcome linked to object of search'].fillna(False)

# according to Dr Wilson's instructions missing values means False 
train['Part of a policing operation'] = train['Part of a policing operation'].fillna(False)

# Outer clothing be filled with False? 
#Yes, except when it’s just a vehicle search, in which case it makes no sense and should be kept as NaN
mask= ((train['Type']!='Vehicle search') & 
      (train['Removal of more than just outer clothing'] != True) &
      (train['Removal of more than just outer clothing'] != False))

train.loc[mask, 'Removal of more than just outer clothing']=False


positive_outcome = ['Local resolution',
           'Community resolution',
           'Arrest',
           'Article found - Detailed outcome unavailable',
           'Caution (simple or conditional)',
           'Khat or Cannabis warning',
           'Offender cautioned',
           'Offender given drugs possession warning',
           'Offender given penalty notice',
           'Penalty Notice for Disorder',
           'Summons / charged by post',
           'Suspect arrested',
           'Suspect summonsed to court',
           'Suspected psychoactive substances seized - No further action']

train['positive_outcome'] = train['Outcome'].isin(positive_outcome)



train['target'] = False
mask=((train["positive_outcome"] == True) & (train["Outcome linked to object of search"]==True))
train.loc[mask, 'target']=True



stations_to_exclude = ['metropolitan', 'gwent', 'humberside']
train_filter = train[~train['station'].isin(stations_to_exclude)]
print("Exclusion of {} observations".format(train[train['station'].isin(stations_to_exclude)]['observation_id'].nunique()))
train_filter.shape

Exclusion of 355849 observations


(304762, 19)

In [7]:
train.groupby(['Outcome linked to object of search','positive_outcome'], dropna=False)['Outcome'].count().unstack()

positive_outcome,False,True
Outcome linked to object of search,,
False,433543,126795
True,37987,62286


### Discrimination

In [3]:
train.groupby(['station','Officer-defined ethnicity'], dropna=False)['Outcome'].count().unstack()

Officer-defined ethnicity,Asian,Black,Mixed,Other,White
station,,,,,
avon-and-somerset,288.0,1468.0,428.0,143.0,7688.0
bedfordshire,974.0,832.0,2.0,59.0,2342.0
btp,757.0,2931.0,NaN,177.0,5690.0
cambridgeshire,67.0,106.0,NaN,23.0,681.0
cheshire,101.0,151.0,NaN,19.0,4574.0
city-of-london,753.0,792.0,NaN,135.0,1895.0
cleveland,111.0,50.0,NaN,24.0,1687.0
cumbria,35.0,24.0,NaN,9.0,2061.0
derbyshire,315.0,281.0,NaN,44.0,2212.0


In [10]:
train.groupby(['station','Gender'], dropna=False)['Outcome'].count().unstack()

Gender,Female,Male,Other
station,,,
avon-and-somerset,1336.0,8679.0,NaN
bedfordshire,379.0,3825.0,5.0
btp,703.0,8849.0,3.0
cambridgeshire,137.0,739.0,1.0
cheshire,429.0,4416.0,NaN
city-of-london,354.0,3218.0,3.0
cleveland,175.0,1697.0,NaN
cumbria,303.0,1825.0,1.0
derbyshire,279.0,2572.0,1.0


In [11]:
train.groupby(['station','target'], dropna=False)['Outcome'].count().unstack()

target,False,True
station,,
avon-and-somerset,7571.0,2444.0
bedfordshire,3396.0,813.0
btp,7455.0,2100.0
cambridgeshire,722.0,155.0
cheshire,3492.0,1353.0
city-of-london,2581.0,994.0
cleveland,1406.0,466.0
cumbria,1647.0,482.0
derbyshire,2149.0,703.0


In [4]:
train.groupby(['station','positive_outcome'], dropna=False)['Outcome'].count().unstack()

positive_outcome,False,True
station,,
avon-and-somerset,6871,3144
bedfordshire,3151,1058
btp,6971,2584
cambridgeshire,541,336
cheshire,2901,1944
city-of-london,2363,1212
cleveland,1272,600
cumbria,1543,586
derbyshire,1955,897


In [14]:
pd.crosstab(train["station"],train["Gender"],values=train["target"], aggfunc='mean')

Gender,Female,Male,Other
station,,,
avon-and-somerset,0.188623,0.252564,NaN
bedfordshire,0.160950,0.196601,0.000000
btp,0.236131,0.218443,0.333333
cambridgeshire,0.182482,0.175913,0.000000
cheshire,0.270396,0.280118,NaN
city-of-london,0.200565,0.286513,0.333333
cleveland,0.200000,0.253978,NaN
cumbria,0.221122,0.227397,0.000000
derbyshire,0.186380,0.253110,0.000000


In [27]:
train_filter.groupby(['Gender','target'], dropna=False)['Outcome'].count().unstack()

target,False,True
Gender,,
Female,25205,5691
Male,217047,56547
Other,224,48


In [20]:
train_filter.groupby(['Officer-defined ethnicity','positive_outcome'], dropna=False)['Outcome'].count().unstack()

positive_outcome,False,True
Officer-defined ethnicity,,
Asian,16858,7740
Black,22167,10176
Mixed,1267,563
Other,4057,1926
White,168556,71452


In [21]:
train_filter.groupby(['Officer-defined ethnicity','target'], dropna=False)['Outcome'].count().unstack()

target,False,True
Officer-defined ethnicity,,
Asian,19626,4972
Black,25596,6747
Mixed,1447,383
Other,4707,1276
White,191100,48908


In [22]:
pd.crosstab(train_filter["station"],train_filter["Officer-defined ethnicity"],values=train_filter["target"], aggfunc='mean')

Officer-defined ethnicity,Asian,Black,Mixed,Other,White
station,,,,,
avon-and-somerset,0.267361,0.219346,0.217290,0.335664,0.247659
bedfordshire,0.212526,0.176683,0.500000,0.101695,0.192997
btp,0.217966,0.174684,NaN,0.220339,0.243234
cambridgeshire,0.328358,0.113208,NaN,0.086957,0.174743
cheshire,0.336634,0.324503,NaN,0.315789,0.276345
city-of-london,0.254980,0.255051,NaN,0.170370,0.304485
cleveland,0.306306,0.220000,NaN,0.291667,0.245406
cumbria,0.257143,0.291667,NaN,0.444444,0.224163
derbyshire,0.307937,0.252669,NaN,0.159091,0.238698


In [24]:
train_filter['Object of search'].value_counts(dropna=False)

Controlled drugs                              192161
Offensive weapons                              35391
Article for use in theft                       30287
Stolen goods                                   26617
Articles for use in criminal damage             6494
Anything to threaten or harm anyone             5241
Firearms                                        2957
Evidence of offences under the Act              1930
Psychoactive substances                         1701
Fireworks                                       1695
Detailed object of search unavailable            129
Game or poaching equipment                        96
Goods on which duty has not been paid etc.        23
Evidence of wildlife offences                     21
Crossbows                                         17
Seals or hunting equipment                         2
Name: Object of search, dtype: int64

In [25]:
pd.crosstab(train_filter["station"],train_filter["Object of search"],values=train_filter["target"], aggfunc='mean')

Object of search,Anything to threaten or harm anyone,Article for use in theft,Articles for use in criminal damage,Controlled drugs,Crossbows,Detailed object of search unavailable,Evidence of offences under the Act,Evidence of wildlife offences,Firearms,Fireworks,Game or poaching equipment,Goods on which duty has not been paid etc.,Offensive weapons,Psychoactive substances,Seals or hunting equipment,Stolen goods
station,,,,,,,,,,,,,,,,
avon-and-somerset,0.058824,0.132075,0.119565,0.296877,0.00,NaN,0.185185,NaN,0.070423,0.000000,0.000000,0.25,0.112163,0.800000,NaN,0.219474
bedfordshire,0.000000,0.067797,0.034483,0.253642,NaN,NaN,NaN,NaN,0.086957,0.111111,NaN,NaN,0.079433,0.000000,NaN,0.164502
btp,0.063465,0.192612,0.333333,0.276552,0.00,NaN,0.117647,1.000000,0.106383,NaN,NaN,0.00,0.087664,0.454545,NaN,0.268357
cambridgeshire,NaN,0.000000,0.000000,0.280943,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.023952,NaN,NaN,0.195122
cheshire,NaN,0.122402,0.218182,0.330009,0.00,NaN,NaN,NaN,0.266667,0.000000,NaN,NaN,0.123641,NaN,NaN,0.359431
city-of-london,0.078947,0.206490,0.074074,0.359896,NaN,NaN,0.182796,NaN,0.114286,0.000000,NaN,NaN,0.125984,0.000000,NaN,0.203488
cleveland,0.256410,0.161392,0.185185,0.286307,NaN,NaN,0.352941,NaN,0.178571,NaN,NaN,NaN,0.215447,NaN,NaN,0.264423
cumbria,NaN,0.119318,0.131579,0.239055,NaN,NaN,NaN,NaN,0.333333,0.000000,0.000000,NaN,0.147186,NaN,NaN,0.336323
derbyshire,NaN,0.167500,0.130435,0.294851,NaN,NaN,0.142857,NaN,0.290323,0.000000,NaN,0.00,0.118421,NaN,NaN,0.224806


In [26]:
train_filter.groupby(['Object of search','target'], dropna=False)['Outcome'].count().unstack()

target,False,True
Object of search,,
Anything to threaten or harm anyone,4866,375
Article for use in theft,27240,3047
Articles for use in criminal damage,6211,283
Controlled drugs,143764,48397
Crossbows,15,2
Detailed object of search unavailable,104,25
Evidence of offences under the Act,1722,208
Evidence of wildlife offences,16,5
Firearms,2645,312


In [4]:
train_filter.groupby(['Age range','target'], dropna=False)['Outcome'].count().unstack()

target,False,True
Age range,,
10-17,51691,8320
18-24,78129,25894
25-34,57724,16150
over 34,54663,11877
under 10,269,45


### Clothing removal

In [9]:
train.groupby(['station','Removal of more than just outer clothing'], dropna=False)['Outcome'].count().unstack()

Removal of more than just outer clothing,False,True,NaN
station,,,
avon-and-somerset,9384.0,631.0,NaN
bedfordshire,4143.0,54.0,12.0
btp,9526.0,29.0,NaN
cambridgeshire,855.0,22.0,NaN
cheshire,4636.0,209.0,NaN
city-of-london,3452.0,123.0,NaN
cleveland,1870.0,NaN,2.0
cumbria,1978.0,146.0,5.0
derbyshire,2626.0,226.0,NaN


In [28]:
train.groupby(['Removal of more than just outer clothing','target'], dropna=False)['Outcome'].count().unstack()

target,False,True
Removal of more than just outer clothing,,
False,589958,59087
True,7483,3182
NaN,884,17


In [29]:
train_filter.groupby(['Removal of more than just outer clothing','target'], dropna=False)['Outcome'].count().unstack()

target,False,True
Removal of more than just outer clothing,,
False,235051,59087
True,7379,3182
NaN,46,17


In [31]:
train_filter.groupby(['Gender','Removal of more than just outer clothing'], dropna=False)['Outcome'].count().unstack()

Removal of more than just outer clothing,False,True,NaN
Gender,,,
Female,29602.0,1287.0,7.0
Male,264276.0,9262.0,56.0
Other,260.0,12.0,NaN


In [34]:
train_filter.groupby(['Age range','Removal of more than just outer clothing'], dropna=False)['Outcome'].count().unstack()

Removal of more than just outer clothing,False,True,NaN
Age range,,,
10-17,59049.0,953.0,9.0
18-24,99972.0,4022.0,29.0
25-34,70862.0,2998.0,14.0
over 34,63953.0,2576.0,11.0
under 10,302.0,12.0,NaN


In [48]:
train_filter.groupby(['Officer-defined ethnicity','Removal of more than just outer clothing'], dropna=False)['Outcome'].count().unstack()

Removal of more than just outer clothing,False,True,NaN
Officer-defined ethnicity,,,
Asian,23619,971,8
Black,30046,2288,9
Mixed,1719,110,1
Other,5769,209,5
White,232985,6983,40


In [33]:
train_filter.groupby(['Age range','Gender','Removal of more than just outer clothing'], dropna=False)['Outcome'].count().unstack()

Removal of more than just outer clothing    False    True   NaN
Age range Gender                                               
10-17     Female                           5720.0    81.0   3.0
          Male                            53261.0   872.0   6.0
          Other                              68.0     NaN   NaN
18-24     Female                           8611.0   315.0   1.0
          Male                            91272.0  3703.0  28.0
          Other                              89.0     4.0   NaN
25-34     Female                           7352.0   405.0   1.0
          Male                            63464.0  2589.0  13.0
          Other                              46.0     4.0   NaN
over 34   Female                           7891.0   482.0   2.0
          Male                            56008.0  2090.0   9.0
          Other                              54.0     4.0   NaN
under 10  Female                             28.0     4.0   NaN
          Male                              271.0     8.0   NaN
          Other                               3.0     NaN   NaN

In [47]:
t = train_filter.groupby(['Age range','Gender','Officer-defined ethnicity','Removal of more than just outer clothing'], dropna=False)['Outcome'].count().unstack()
t.tail(30)

Removal of more than just outer clothing      False    True  NaN
Age range Gender Officer-defined ethnicity                      
25-34     Male   Other                       1326.0    54.0  NaN
                 White                      49634.0  1728.0  9.0
          Other  Asian                          6.0     NaN  NaN
                 Black                          4.0     1.0  NaN
                 Other                          1.0     NaN  NaN
                 White                         35.0     3.0  NaN
over 34   Female Asian                        126.0     8.0  NaN
                 Black                        343.0    15.0  NaN
                 Mixed                         24.0     4.0  NaN
                 Other                         90.0     6.0  NaN
                 White                       7308.0   449.0  2.0
          Male   Asian                       2831.0   108.0  NaN
                 Black                       4237.0   267.0  3.0
                 Mixed                        183.0     8.0  NaN
                 Other                        851.0    27.0  1.0
                 White                      47906.0  1680.0  5.0
          Other  Asian                          3.0     NaN  NaN
                 Black                          4.0     NaN  NaN
                 Other                          3.0     NaN  NaN
                 White                         44.0     4.0  NaN
under 10  Female Black                          2.0     NaN  NaN
                 Other                          1.0     NaN  NaN
                 White                         25.0     4.0  NaN
          Male   Asian                         14.0     1.0  NaN
                 Black                         41.0     1.0  NaN
                 Mixed                          2.0     NaN  NaN
                 Other                          6.0     NaN  NaN
                 White                        208.0     6.0  NaN
          Other  Other                          1.0     NaN  NaN
                 White                          2.0     NaN  NaN

In [30]:
train_filter.groupby(['station','Removal of more than just outer clothing'], dropna=False)['Outcome'].count().unstack()

Removal of more than just outer clothing,False,True,NaN
station,,,
avon-and-somerset,9384.0,631.0,NaN
bedfordshire,4143.0,54.0,12.0
btp,9526.0,29.0,NaN
cambridgeshire,855.0,22.0,NaN
cheshire,4636.0,209.0,NaN
city-of-london,3452.0,123.0,NaN
cleveland,1870.0,NaN,2.0
cumbria,1978.0,146.0,5.0
derbyshire,2626.0,226.0,NaN
